# A1: Implementação dos exercícios de modelagem de PLI

### Aluna: Sofia Lakschevitz
### Professor: Vincent Gerard Y. Guigues

## Importando a biblioteca Mosek

In [46]:
import mosek
import mosek.fusion as mf
import mosek.fusion.pythonic
import pandas as pd

## List 5

### Exercise 1

In [85]:
# Transportation Problem: List 5 Exercise 1

with mf.Model("list5_exercise1") as M51:
    # Indexes
    factories = [0, 1]  # U1, U2
    clients = [0, 1, 2] # E1, E2, E3

    #Paremeter
    demands = [100, 200, 300]
    capacities = [400, 300]

    # Decision variables
    # x[i,j] = amount of product transported from factory i to client j
    # x[i,j] >= 0 and x is an integer
    x = M51.variable("x", [len(factories), len(clients)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Costs matrix
    costs = [[1.0, 1.5, 3.5],
             [2.0, 1.0, 2.0]]

    # Ojective function: minimize total cost
    M51.objective("MinimizeCost", mf.ObjectiveSense.Minimize, mf.Expr.dot(costs, x))

    # Constraint: demand for each client
    for j in clients:
        M51.constraint(f"Demand{j+1}", mf.Expr.add([x.index(i, j) for i in factories]), mf.Domain.equalsTo(demands[j]))

    # Constraint: capacity for each factory
    for i in factories:
        M51.constraint(f"Capacity{i+1}", mf.Expr.add([x.index(i, j) for j in clients]), mf.Domain.lessThan(capacities[i]))

    # Constraint: non-negativity of decision variables
    # (already included in the variable definition)

    # Solving the model
    M51.solve()

    solution_status = M51.getPrimalSolutionStatus()
    minimal_cost = M51.primalObjValue()

    # Display solution
    print("Solution status:", solution_status)

    print("\nMinimal cost:", minimal_cost)

    print("\nOptimal quantities x[i,j] of product to be transported from factory i to client j:")
    for i in factories:
        for j in clients:
            print(f"x[{i+1},{j+1}] = {x.index(i,j).level()[0]}")

    print("\nExcedents in factories:")
    for i in factories:
        print(f"Excedent Factory {i+1} = {capacities[i] - (x.index(i, 0).level()[0] + x.index(i, 1).level()[0] + x.index(i, 2).level()[0])}")


Solution status: SolutionStatus.Optimal

Minimal cost: 1000.0

Optimal quantities x[i,j] of product to be transported from factory i to client j:
x[1,1] = 100.0
x[1,2] = 200.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 300.0

Excedents in factories:
Excedent Factory 1 = 100.0
Excedent Factory 2 = 0.0


### Exercise 2

#### Part 1

In [72]:
# Mixture Problem: List 5 Exercise 2

with mf.Model("list5_exercise2") as M52:
    # Indexes
    products = [0, 1] # P1, P2
    fertilizers = [0, 1, 2] # Ni, KCl, P

    # Parameters
    # Product prices
    product_prices = [7, 9] # in $/kg
    # Fertilizer supply
    fertilizer_supply = [8, 19, 4] # in $/kg
    # Fertilizer contents in products
    fertilizer_contents = [[1, 2], # Fertilizer Ni (1)
                           [1, 3], # Fertilizer KCl (2)
                           [0, 1]] # Fertilizer P (3)

    # Decision variables measured in kilograms
    # p[j] = amount of product j produced
    # p[j] >= 0 and p[j] is an integer
    p = M52.variable("p", [len(products)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    #Objective function: maximize profit
    M52.objective("MaximizeProfit", mf.ObjectiveSense.Maximize, mf.Expr.dot(product_prices, p))

    # Constraints: fertilizer content in products
    for i in fertilizers:
        M52.constraint(f"Fertilizer{i+1}", mf.Expr.dot(fertilizer_contents[i], p), mf.Domain.lessThan(fertilizer_supply[i]))

    # Constraints: non-negativity of decision variables
    for j in products:
        M52.constraint(f"NonNegativity{j+1}", p.index(j), mf.Domain.greaterThan(0.0))

    # Solving the model
    M52.solve()

    solution_status = M52.getPrimalSolutionStatus()
    max_profit = M52.primalObjValue()

    # Display solution
    print("Solution status:", solution_status)
    print("\nMaximum profit:", max_profit)
    print("\nOptimal quantities p[j] of product j to be produced:")
    for j in products:
        print(f"p[{j+1}] = {p.level()[j]}")
    print("\nExcedents in fertilizers:")
    for i in fertilizers:
        print(f"Excedent Fertilizer {i+1} = {fertilizer_supply[i] - (p.index(0).level()[0] * fertilizer_contents[i][0] + p.index(1).level()[0] * fertilizer_contents[i][1])}")


Solution status: SolutionStatus.Optimal

Maximum profit: 56.0

Optimal quantities p[j] of product j to be produced:
p[1] = 8.0
p[2] = 0.0

Excedents in fertilizers:
Excedent Fertilizer 1 = 0.0
Excedent Fertilizer 2 = 11.0
Excedent Fertilizer 3 = 4.0


#### Part 2

In [82]:
with mf.Model("list5_exercise22") as M522:
    # Indexes
    products = [0, 1] # P1, P2
    fertilizers = [0, 1, 2] # Ni, KCl, P

    # Parameters
    # Product prices
    product_prices = [7, 9] # in $/kg
    # Fertilizer supply
    fertilizer_supply = [8, 19, 4] # in $/kg
    # Fertilizer contents in products
    fertilizer_contents = [[1, 2], # Fertilizer Ni (1)
                           [1, 3], # Fertilizer KCl (2)
                           [0, 1]] # Fertilizer P (3)

    # Decision variables
    # y[i] = price of 1kg of fertilizer i
    # y[i] >= 0 and y[i] is an integer
    y = M522.variable("y", [len(fertilizers)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    #Objective function: minimize cost
    M522.objective("MinimizeCost", mf.ObjectiveSense.Minimize, mf.Expr.dot(y, fertilizer_supply))

    # Constraints: selling must be as profitable as producing
    #for j in products:
    #    M522.constraint(f"Product{j+1}", mf.Expr.dot(y,fertilizer_contents[j]), mf.Domain.greaterThan(product_prices[j]))

    for j in products:
        M522.constraint(f"Product_{j+1}", mf.Expr.dot(y, [fertilizer_contents[i][j] for i in fertilizers]), mf.Domain.greaterThan(product_prices[j]))

    # Constraints: non-negativity of decision variables
    for i in fertilizers:
        M522.constraint(f"NonNegativity{i+1}", y.index(i), mf.Domain.greaterThan(0.0))

    # Solving the model
    M522.solve()

    solution_status = M522.getPrimalSolutionStatus()
    min_cost = M522.primalObjValue()

    # Display solution
    print("Solution status:", solution_status)
    print("\nMinimal cost:", min_cost)
    print("\nOptimal prices y[i] of fertilizer i:")
    for i in fertilizers:
        print(f"y[{i+1}] = {y.level()[i]}")
    

Solution status: SolutionStatus.Optimal

Minimal cost: 56.0

Optimal prices y[i] of fertilizer i:
y[1] = 7.0
y[2] = 0.0
y[3] = 0.0


### Exercise 3

In [87]:
# Transportation Problem - List 5 Exercise 3

with mf.Model("list5_exercise3") as M53:
    # Indexes
    u_factories = [0, 1]  # U1, U2
    v_factories = [0, 1, 2] # V1, V2, V3

    # Parameters
    demands = [5, 4, 3]
    capacities = [6, 6]

    # Decision variables x[i,j] >= 0 and x is an integer
    # x[i,j] = amount of engines transported from u_factory i to v_factory j
    x = M53.variable("x", [len(u_factories), len(v_factories)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Costs of transport matrix
    # costs[i][j] = cost of transporting from u_factory i to v_factory j
    costs = [[38, 27, 48],
             [37, 58, 45]]

    # Ojective function: minimize total cost
    M53.objective("MinimizeCost", mf.ObjectiveSense.Minimize, mf.Expr.dot(costs, x))

    # Constraint: demand for each v_factory j
    for j in v_factories:
        M53.constraint(f"Demand{j+1}", mf.Expr.add([x.index(i, j) for i in u_factories]), mf.Domain.equalsTo(demands[j]))

    # Constraint: capacity for each factory
    for i in u_factories:
        M53.constraint(f"Capacity{i+1}", mf.Expr.add([x.index(i, j) for j in v_factories]), mf.Domain.lessThan(capacities[i]))

    # Solving the model
    M53.solve()
    solution_status = M53.getPrimalSolutionStatus()
    minimal_cost = M53.primalObjValue()

    # Display solution
    print("Solution status:", solution_status)

    print("\nMinimal cost:", minimal_cost)

    print("\nOptimal quantities of engines to be transported from city U factory i to city V factory j:")
    for i in u_factories:
        for j in v_factories:
            print(f"x[{i+1},{j+1}] = {x.index(i,j).level()[0]}")

    print("\nExcedents in the factories from city U:")
    for i in u_factories:
        print(f"Excedent U Factory {i+1} = {capacities[i] - (x.index(i, 0).level()[0] + x.index(i, 1).level()[0] + x.index(i, 2).level()[0])}")

Solution status: SolutionStatus.Optimal

Minimal cost: 430.0

Optimal quantities of engines to be transported from city U factory i to city V factory j:
x[1,1] = 2.0
x[1,2] = 4.0
x[1,3] = 0.0
x[2,1] = 3.0
x[2,2] = 0.0
x[2,3] = 3.0

Excedents in the factories from city U:
Excedent U Factory 1 = 0.0
Excedent U Factory 2 = 0.0


### Exercise 4

In [88]:
# Mixture Problem - List 5 Exercise 4

with mf.Model("list5_exercise4") as M54:
    # Decision variables s,c >= 0 and s,c are integers
    s = M54.variable("s", 1, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    c = M54.variable("c", 1, mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Ojective function: maximize profit
    M54.objective("MaximizeProfit", mf.ObjectiveSense.Maximize, 5*s + 4*c)

    # Constraint: supply of hours
    M54.constraint(f"HoursSupply", (1/6)*s + (1/5)*c, mf.Domain.lessThan(10))

    # Constraint: supply of leather
    M54.constraint(f"LeatherSupply", 2*s + c, mf.Domain.lessThan(78))

    # Solving the model
    M54.solve()
    solution_status = M54.getPrimalSolutionStatus()
    maximal_profit = M54.primalObjValue()
    optimal_s = int(round(s.level()[0]))
    optimal_c = int(round(c.level()[0]))

    # Display solution
    print("Solution status:", solution_status)

    print("\nMaximal profit:", maximal_profit)

    print("\nOptimal quantities of shoes and belts to be producted in one day by the facturer:")
    print(f"s = {optimal_s}")
    print(f"c = {optimal_c}")

    print("\nHours of work:")
    print(f"Quantity of hours spent on shoes = {(1/6)*optimal_s} hours")
    print(f"Quantity of hours spent on belts = {(1/5)*optimal_c} hours")

Solution status: SolutionStatus.Optimal

Maximal profit: 240.0

Optimal quantities of shoes and belts to be producted in one day by the facturer:
s = 24
c = 30

Hours of work:
Quantity of hours spent on shoes = 4.0 hours
Quantity of hours spent on belts = 6.0 hours


### Exercise 5

In [89]:
# Mixture Problem - List 5 Exercise 5

with mf.Model("list5_exercise5") as M55:
    # Indexes
    diesel_types = [0, 1] # Diesel A, Diesel B
    gasoline_types = [0, 1, 2] # Gasoline 1, Gasoline 2, Gasoline 3

    # Paremeter
    diesel_prices = [20, 30] # in $/gallon
    gasoline_supply = [500, 200, 200] # in gallons
    gasoline_contents = [[0.25, 0.0], # Gasoline 1 (1)
                         [0.25, 0.5], # Gasoline 2 (2)
                         [0.5, 0.5]] # Gasoline 3 (3)

    #Decision Variables
    # d[j] >= 0 and d[j] is an integer
    d = M55.variable("d", [len(diesel_types)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Ojective function: maximize profit
    M55.objective("MaximizeProfit", mf.ObjectiveSense.Maximize, mf.Expr.dot(diesel_prices, d))

    # Constraints: gasoline content in diesel
    for i in gasoline_types:
        M55.constraint(f"Gasoline{i+1}", mf.Expr.dot(gasoline_contents[i], d), mf.Domain.lessThan(gasoline_supply[i]))

    # Constraints: non-negativity of decision variables
    for j in diesel_types:
        M55.constraint(f"NonNegativity{j+1}", d.index(j), mf.Domain.greaterThan(0.0))

    # Solving the model
    M55.solve()
    solution_status = M55.getPrimalSolutionStatus()
    maximal_profit = M55.primalObjValue()

    # Display solution
    print("Solution status:", solution_status)
    
    print("\nMaximal profit:", maximal_profit)

    print("\nOptimal quantities of diesel A and diesel B to be produced:")
    for j in diesel_types:
        print(f"d[{j+1}] = {d.level()[j]}")
    print("\nAmount of gallons used for each type of gasoline:")
    for i in gasoline_types:
        print(f"Gasoline {i+1} = {sum(gasoline_contents[i][j] * d.level()[j] for j in diesel_types)} gallons")



Solution status: SolutionStatus.Optimal

Maximal profit: 12000.0

Optimal quantities of diesel A and diesel B to be produced:
d[1] = 0.0
d[2] = 400.0

Amount of gallons used for each type of gasoline:
Gasoline 1 = 0.0 gallons
Gasoline 2 = 200.0 gallons
Gasoline 3 = 200.0 gallons


### Exercise 6

In [93]:
# Mixture Problem - List 5 Exercise 6

with mf.Model("list5_exercise6") as M56:
    #indexes
    diesel_types = [0, 1, 2]  # Diesel types
    oil_types = [0, 1, 2, 3]  # Oil types

    # Paremeters
    diesel_prices = [5.5, 4.5, 3.5]  # Prices of diesel types
    oil_prices = [3, 6, 4, 5]  # Prices of oil types
    oil_supply = [3000, 2000, 4000, 1000]  # Supply of each oil types

    # Decision Variables
    # x[i,j] = amount of oil type i used to produce diesel type j
    # x[i,j] >= 0 and x[i,j] is an integer
    x = M56.variable("x", [len(oil_types), len(diesel_types)], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Objective function: maximize profit
    revenue = sum(diesel_prices[j] * sum(x.index(i, j) for i in oil_types) for j in diesel_types)
    costs = sum(oil_prices[i] * sum(x.index(i, j) for j in diesel_types) for i in oil_types)

    M56.objective("MaximizeProfit", mf.ObjectiveSense.Maximize, revenue - costs)

    # Constraint: diesel composition
    # Diesel A
    M56.constraint("DemandA1", mf.Expr.sub(x.index(0,0), 0.3 * mf.Expr.add([x.index(i, 0) for i in oil_types])), mf.Domain.lessThan(0.0))
    M56.constraint("DemandA2", mf.Expr.sub(0.4 * mf.Expr.add([x.index(i, 0) for i in oil_types]), x.index(1,0)), mf.Domain.lessThan(0.0))
    M56.constraint("DemandA3", mf.Expr.sub(x.index(2,0), 0.5 * mf.Expr.add([x.index(i, 0) for i in oil_types])), mf.Domain.lessThan(0.0))

    # Diesel B
    M56.constraint("DemandB1", mf.Expr.sub(x.index(0,1), 0.5 * mf.Expr.add([x.index(i, 1) for i in oil_types])), mf.Domain.lessThan(0.0))
    M56.constraint("DemandB2", mf.Expr.sub(0.1 * mf.Expr.add([x.index(i, 1) for i in oil_types]), x.index(1,1)), mf.Domain.lessThan(0.0))

    # Diesel C
    M56.constraint("DemandC1", mf.Expr.sub(x.index(0,2), 0.7 * mf.Expr.add([x.index(i, 2) for i in oil_types])), mf.Domain.lessThan(0.0))

    # Constraint: oil supply
    for i in oil_types:
        M56.constraint(f"OilSupply{i+1}", mf.Expr.add([x.index(i, j) for j in diesel_types]), mf.Domain.lessThan(oil_supply[i]))

    # Solving the model
    M56.solve()

    solution_status = M56.getPrimalSolutionStatus()
    maximal_profit = M56.primalObjValue()
    optimal_x = [[int(round(x.index(i,j).level()[0])) for j in diesel_types] for i in oil_types]

    # Display solution
    print("Solution status:", solution_status)
    print("\nMaximal profit:", maximal_profit)
    print("\nOptimal quantities of oil types to be used for each diesel type:")
    for i in oil_types:
        for j in diesel_types:
            print(f"x[{i+1},{j+1}] = {optimal_x[i][j]}")
    print("\nQuantities of diesel types produced:")
    for j in diesel_types:
        print(f"Diesel {j+1} = {sum(optimal_x[i][j] for i in oil_types)} gallons")

Solution status: SolutionStatus.Optimal

Maximal profit: 7165.500000000003

Optimal quantities of oil types to be used for each diesel type:
x[1,1] = 1099
x[1,2] = 1901
x[1,3] = 0
x[2,1] = 1467
x[2,2] = 533
x[2,3] = 0
x[3,1] = 1101
x[3,2] = 2896
x[3,3] = 0
x[4,1] = 0
x[4,2] = 0
x[4,3] = 0

Quantities of diesel types produced:
Diesel 1 = 3667 gallons
Diesel 2 = 5330 gallons
Diesel 3 = 0 gallons


## List 6

### Exercise 1

In [122]:
# Electricity Production Management - List 6 Exercise 1

with mf.Model('list6_exercise1') as M61:
    # Parameters
    # Number of days
    T = 30
    # Number of regions
    regions = [0, 1, 2, 3]
    # Thermal plants per region
    thermal_plants = {0: [0, 1], 1: [0], 2: [0, 1, 2], 3: [0]}
    # Energy exchange graph
    G = [(0, 1), (1, 0), (1, 2), (2, 3), (3, 0)]
    # Inflows for each day and region
    A = [[100]*4 for _ in range(T)]
    # Demand
    D = [[500]*4 for _ in range(T)]
    # Thermal unit costs
    c = {0: [50, 60], 1: [70], 2: [40, 45, 55], 3: [65]}
    # Spot prices
    p = [100] * T
    # Thermal capacities
    u_max = {0: [200, 150], 1: [180], 2: [220, 210, 200], 3: [190]}
    # Hydro capacities
    v_max = [300, 250, 350, 280]
    # Exchange capacities
    E_max = {(i,j): 200 for (i,j) in G}
    # Min reservoir levels
    x_min = [1000, 1200, 900, 1100]
    # Max reservoir levels
    x_max = [5000, 5500, 6000, 5200]
    # Initial reservoir levels
    initial_x = [2000, 2500, 3000, 2400]

    # Precompute incoming and outgoing edges for each region
    incoming = {i: [j for (j, k) in G if k == i] for i in regions}
    outgoing = {i: [k for (j, k) in G if j == i] for i in regions}

    # Decision variables
    # Thermal production
    # u[t,i,j] = amount of energy produced by thermal plant j in region i at time t
    # u[t,i,j] >= 0 and u[t,i,j] <= u_max[i][j] and u[t,i,j] is an integer
    u = {(t, i, j): M61.variable(f'u_{t}_{i}_{j}', mf.Domain.integral(mf.Domain.inRange(0.0, u_max[i][j])))
            for t in range(T) for i in regions for j in thermal_plants[i]}
    # Hydro production
    # v[t,i] = amount of energy produced by hydro plant in region i at time t
    # v[t,i] >= 0 and v[t,i] <= v_max[i] and v[t,i] is an integer
    v = {(t, i): M61.variable(f'v_{t}_{i}', mf.Domain.integral(mf.Domain.inRange(0.0, v_max[i])))
            for t in range(T) for i in regions}
    # Spot purchases
    # s[t,i] = amount of energy purchased in region i at time t
    # s[t,i] >= 0 and s[t,i] is an integer
    s = {(t, i): M61.variable(f's_{t}_{i}', mf.Domain.integral(mf.Domain.greaterThan(0.0)))
            for t in range(T) for i in regions}
    # Energy exchange
    # E[t,i,j] = amount of energy exchanged from region i to region j at time t
    # E[t,i,j] >= 0 and E[t,i,j] <= E_max[i][j] and E[t,i,j] is an integer
    E = {(t, i, j): M61.variable(f'E_{t}_{i}_{j}', mf.Domain.integral(mf.Domain.inRange(0.0, E_max[(i,j)])))
            for t in range(T) for (i,j) in G}
    # Reservoir levels (T+1 days)
    # x[t,i] = amount of energy stored in reservoir i at time t
    # x[t,i] >= x_min[i] and x[t,i] <= x_max[i] and x[t,i] is an integer
    x = {(t, i): M61.variable(f'x_{t}_{i}', mf.Domain.integral(mf.Domain.inRange(x_min[i], x_max[i])))
            for t in range(T+1) for i in regions}

    # Objective function: Minimize total cost
    obj_terms = []
    # Thermal production costs
    for (t, i, j) in u:
        obj_terms.append(mf.Expr.mul(c[i][j], u[(t, i, j)]))
    # Spot purchase costs
    for (t, i) in s:
        obj_terms.append(mf.Expr.mul(p[t], s[(t, i)]))

    M61.objective(mf.ObjectiveSense.Minimize, mf.Expr.add(obj_terms))

    # Constraint: Reservoir balance: x_{t+1,i} = x_{t,i} + 0.8*A[t][i] - v_{t,i}
    for t in range(T):
        for i in regions:
            M61.constraint(f'reservoir_{t}_{i}', 
                           mf.Expr.sub(x[(t+1, i)], mf.Expr.add(x[(t, i)], 
                                                                mf.Expr.sub(mf.Expr.constTerm(0.8 * A[t][i]), v[(t, i)]))),
                           mf.Domain.equalsTo(0.0))

    # Constraint: Initial reservoir levels
    for i in regions:
        M61.constraint(f'initial_x_{i}', x[(0, i)], mf.Domain.equalsTo(initial_x[i]))

    # Constraint: Demand satisfaction
    for t in range(T):
        for i in regions:
            # Thermal production sum
            thermal_sum = mf.Expr.add([u[(t, i, j)] for j in thermal_plants[i]] if thermal_plants[i] else [0.0])
            # Energy exchange terms
            sum_in = mf.Expr.add([E[(t, j, i)] for j in incoming[i]] if incoming[i] else [0.0])
            sum_out = mf.Expr.add([E[(t, i, k)] for k in outgoing[i]] if outgoing[i] else [0.0])
            # Total supply expression
            supply = mf.Expr.add([
                mf.Expr.constTerm(0.2 * A[t][i]),
                thermal_sum,
                v[(t, i)],
                s[(t, i)],
                sum_in,
                mf.Expr.neg(sum_out)
            ])
            M61.constraint(f'demand_{t}_{i}', supply, mf.Domain.greaterThan(D[t][i]))

    # Solving the model
    M61.solve()

    solution_status = M61.getPrimalSolutionStatus()
    minimal_cost = M61.primalObjValue()

    # Display solution
    print("Solution status:", solution_status)

    print("\nMinimal cost:", minimal_cost)

    print("\nOptimal quantities of energy produced by thermal plants:")
    for (t, i, j) in u:
        print(f"u[{t},{i},{j}] = {int(round(u[(t,i,j)].level()[0]))}")

    print("\nOptimal quantities of energy produced by hydro plants:")
    for (t, i) in v:
        print(f"v[{t},{i}] = {int(round(v[(t,i)].level()[0]))}")

    print("\nOptimal quantities of energy purchased:")
    for (t, i) in s:
        print(f"s[{t},{i}] = {int(round(s[(t,i)].level()[0]))}")

    print("\nOptimal quantities of energy exchanged:")
    for (t, i, j) in E:
        print(f"E[{t},{i},{j}] = {int(round(E[(t,i,j)].level()[0]))}")

    print("\nOptimal reservoir levels:")
    for (t, i) in x:
        print(f"x[{t},{i}] = {int(round(x[(t,i)].level()[0]))}")    


Solution status: SolutionStatus.Optimal

Minimal cost: 2511000.0

Optimal quantities of energy produced by thermal plants:
u[0,0,0] = 200
u[0,0,1] = 150
u[0,1,0] = 180
u[0,2,0] = 220
u[0,2,1] = 210
u[0,2,2] = 0
u[0,3,0] = 190
u[1,0,0] = 200
u[1,0,1] = 150
u[1,1,0] = 180
u[1,2,0] = 220
u[1,2,1] = 210
u[1,2,2] = 200
u[1,3,0] = 190
u[2,0,0] = 200
u[2,0,1] = 150
u[2,1,0] = 180
u[2,2,0] = 220
u[2,2,1] = 210
u[2,2,2] = 200
u[2,3,0] = 190
u[3,0,0] = 200
u[3,0,1] = 150
u[3,1,0] = 180
u[3,2,0] = 220
u[3,2,1] = 210
u[3,2,2] = 0
u[3,3,0] = 190
u[4,0,0] = 200
u[4,0,1] = 150
u[4,1,0] = 180
u[4,2,0] = 220
u[4,2,1] = 210
u[4,2,2] = 200
u[4,3,0] = 190
u[5,0,0] = 200
u[5,0,1] = 150
u[5,1,0] = 180
u[5,2,0] = 220
u[5,2,1] = 210
u[5,2,2] = 0
u[5,3,0] = 190
u[6,0,0] = 200
u[6,0,1] = 150
u[6,1,0] = 180
u[6,2,0] = 220
u[6,2,1] = 210
u[6,2,2] = 200
u[6,3,0] = 190
u[7,0,0] = 200
u[7,0,1] = 150
u[7,1,0] = 180
u[7,2,0] = 220
u[7,2,1] = 210
u[7,2,2] = 0
u[7,3,0] = 190
u[8,0,0] = 200
u[8,0,1] = 150
u[8,1,0] = 180


### Exercise 2

In [94]:
# Asset Management - List 6 Exercise 2

with mf.Model("list6_exercise2") as M62:
    # Parameters
    # Amount of money available for investment
    M = 1000000
    # Number of assets
    N = 5
    # Expected returns of each asset
    R = [0.1, 0.2, 0.15, 0.17, 0.18]

    # Decision variables
    # x[i] = amount of money invested in asset i
    # x[i] >= 0 and x is an integer 
    x = M62.variable("x", [N], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Objective function: maximize total return
    M62.objective("MaximizeReturn", mf.ObjectiveSense.Maximize, mf.Expr.dot(R, x))

    # Constraint: total investment
    M62.constraint("TotalInvestment", mf.Expr.sum(x), mf.Domain.lessThan(M))

    # Solving the model
    M62.solve()

    solution_status = M62.getPrimalSolutionStatus()
    maximal_return = M62.primalObjValue()
    optimal_x = [int(round(x.index(i).level()[0])) for i in range(N)]

    # Display solution
    print("Solution status:", solution_status)
    print("\nMaximal return:", maximal_return)
    print("\nOptimal amounts to be invested in each asset:")
    for i in range(N):
        print(f"x[{i+1}] = {optimal_x[i]}")
    print("\nTotal investment:", sum(optimal_x))

Solution status: SolutionStatus.Optimal

Maximal return: 200000.0

Optimal amounts to be invested in each asset:
x[1] = 0
x[2] = 1000000
x[3] = 0
x[4] = 0
x[5] = 0

Total investment: 1000000


### Exercise 3

In [ ]:
# Production Expansion Planning - List 6 Exercise 3

with mf.Model('list6_exercise3') as M63:
    # Paremeters
    # Number of products
    m = 3
    # Number of machines
    n = 4
    # Maximum of hours of revision for each machine in a week
    T = 100
    # Costs of each unit not sold of each product
    p = [400, 400, 400]  # Penalty for unsold products
    # Cost per extra hour for each machine
    c = [2.5, 3.75, 5.0, 3.0]
    # Revision time per hour for each machine (in hours)
    t = [0.08, 0.04, 0.03, 0.01]
    # Regular hours available for each machine
    h_j = [500] * n
    # Upper limits for machine hours
    u = [2000, 2000, 3000, 3000]
    # Production rate matrix (products x machines)
    a = [[0.6, 0.6, 0.9, 0.8],
         [0.1, 0.9, 0.6, 0.8],
         [0.05, 0.2, 0.5, 0.8]]
    # Production cost matrix (products x machines)
    g = [[2.6, 3.4, 3.4, 2.5],
         [1.5, 2.4, 2.0, 3.6],
         [4.0, 3.8, 3.5, 3.2]]
    # Demand for each product
    demand = [1800, 600, 3000]
    # Upper bounds for extra hours e_j
    e_upper = [u[j] - h_j[j] for j in range(n)]

    # Decision Variables
    # h[i][j]: hours of machine j used for product i
    # h[i][j] >= 0 and h[i][j] is an integer
    h = M63.variable('h', [m, n], mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    # e[j]: extra hours for machine j, with upper bounds
    # e[j] >= 0 and e[j] <= e_upper[j] and e[j] is an integer
    e = M63.variable('e', n, mf.Domain.integral(mf.Domain.inRange(0.0, e_upper)))
    # s[i]: unsold quantity of product i
    s = M63.variable('s', m, mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Objective: Minimize total cost
    obj = mf.Expr.add([
        mf.Expr.sum(mf.Expr.mulElm(g, h)),  # Production cost
        mf.Expr.dot(c, e),               # Extra hours cost
        mf.Expr.dot(p, s)                 # Unsold penalty
    ])
    M63.objective('MinimizeCost', mf.ObjectiveSense.Minimize, obj)

    # Constraint: Total revision time <= T
    # sum(t_j * (h_j + e_j)) <= 100 → sum(t_j * e_j) <= 20
    M63.constraint('RevisionTime', mf.Expr.dot(t, e), mf.Domain.lessThan(20.0))

    # Constraint: Demand satisfaction: x_i + s_i = d_i
    for i in range(m):
        x_i = mf.Expr.dot(a[i], h.slice([i, 0], [i+1, n]))
        M63.constraint(f'Demand_{i}', mf.Expr.add(x_i, s.index(i)), mf.Domain.equalsTo(demand[i]))

    # Constraint: Machine hours: sum_i h[i][j] = h_j[j] + e_j
    for j in range(n):
        sum_h = mf.Expr.sum(h.slice([0, j], [m, j+1]))
        M63.constraint(f'MachineHours_{j}', mf.Expr.sub(sum_h, e.index(j)), mf.Domain.equalsTo(h_j[j]))

    # Solving the model
    M63.solve()

    solution_status = M63.getPrimalSolutionStatus()
    minimal_cost = round(M63.primalObjValue())

    # Display solution
    print(f"Solution status: {solution_status}")
    print(f"\nMinimal cost: {minimal_cost}")

    # Retrieve variable values
    e_values = e.level()
    s_values = s.level()
    h_values = h.level()

    print("\nExtra hours (e):")
    for j in range(n):
        print(f"Machine {j+1}: {e_values[j]:.2f} hours")

    print("\nUnsold products (s):")
    for i in range(m):
        print(f"Product {i+1}: {s_values[i]:.2f} units")

    print("\nProduction hours (h):")
    for i in range(m):
        row = h_values[i*n : (i+1)*n]
        print(f"Product {i+1}: {[f'{hr:.2f}' for hr in row]} hours")


Solution status: SolutionStatus.Optimal

Minimal cost: 897283

Extra hours (e):
Machine 1: 0.00 hours
Machine 2: 0.00 hours
Machine 3: 0.00 hours
Machine 4: 2000.00 hours

Unsold products (s):
Product 1: 2.00 units
Product 2: 150.00 units
Product 3: 2048.00 units

Production hours (h):
Product 1: ['500.00', '0.00', '500.00', '1310.00'] hours
Product 2: ['0.00', '500.00', '0.00', '0.00'] hours
Product 3: ['0.00', '0.00', '0.00', '1190.00'] hours


### Exercise 4

In [130]:
# Contract Management with Cancellation Option - List 6 Exercise 4

with mf.Model("list6_exercise4") as M64:
    # Parameters
    # Number of months
    T = 12
    # Number of contracts
    N = 3
    # Spot price for month t
    S_t = [100, 105, 110, 108, 115, 120, 118, 122, 125, 130, 128, 135]
    # Quantity of contracted gas for t, one part of it, u^(0,t)_p, being injected in the pipeline and the rest u^(1,t)_p being injectes in the storage
    Q_t = [500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610]
    # Cost of unit storage at t
    c_t = [0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35]
    # Cost of unsatisfied demand at t
    p_t = [60] * T
    # Demand at t
    D_t = [450, 470, 490, 500, 520, 540, 560, 580, 600, 620, 640, 660]
    # Arrival time of the ship of contract n
    t_n = [4, 8, 11]
    # load of contract n
    Q_n = [300, 320, 340]
    # cancellation fee paid at t on contract n
    f_nt = [[0.2 * (t_n[n] - t) if t <= t_n[n] - 1 else 0 for t in range(T)] for n in range(N)]

    # Decision variables
    # Quantity of gas injected in the storage
    u_s = M64.variable("u_s", T, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    # Quantity of gas injected in the pipeline
    u_p = M64.variable("u_p", T, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    # Quantity of gas transferred from the pipeline to the clients at t
    u_ps = M64.variable("u_ps", T, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    # Quantity of unsatisfied demand at t
    u_NS = M64.variable("u_NS", T, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    # Pipeline storage at the beginning of time t
    s_p = M64.variable("s_p", T+1, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    # Stock in ship of contract n at t >= t_n-1
    s_n = M64.variable("s_n", [N, T], mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    # 1 if contract n is cancelled at t or before and 0 otherwise
    y = M64.variable("y", [N, T], mf.Domain.integral(mf.Domain.binary()))
    # if contract n is cancelled at t or before, y_n^t = 1, else y_n^t = 0
    x = M64.variable("x", N, mf.Domain.binary())
    # Cancellation cost at t for contract n
    Cn = M64.variable("Cn", N, mf.Domain.integral(mf.Domain.greaterThan(0.0)))
    # Ship load n cost at t
    Fn = M64.variable("Fn", N, mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Objective function: Minimize total cost
    # Part 1: acumulated costs 
    monthly_cost = None
    for t in range(T):
        storage_cost = c_t[t] * mf.Expr.add(s_p.index(t), mf.Expr.sum(s_n.slice([0, t], [N, t+1])))
        penalty_cost = p_t[t] * u_NS.index(t)
        sale_revenue = 1.35 * S_t[t] * u_p.index(t)
        total_month_cost = mf.Expr.sub(mf.Expr.add(storage_cost, penalty_cost), sale_revenue)

        if monthly_cost is None:
            monthly_cost = total_month_cost
        else:
            monthly_cost = mf.Expr.add(monthly_cost, total_month_cost)

    # Part 2: cancellation cost + cost of ships arrival
    total_cancel_arrival = mf.Expr.add(mf.Expr.sum(Cn), mf.Expr.sum(Fn))

    # Final objective function
    expr_cost = mf.Expr.add(monthly_cost, total_cancel_arrival)

    M64.objective("Minimize Total Cost", mf.ObjectiveSense.Minimize, expr_cost)


    # Constraints
    for t in range(T):
        # Demand: u_ps + u_NS = D_t
        M64.constraint(f"demand_balance_{t}", 
                     mf.Expr.sub(D_t[t], mf.Expr.add(u_ps.index(t), u_NS.index(t))) == 0)

        # Estoque no gasoduto: s_{t+1} = s_t + u_s - u_ps
        M64.constraint(f"pipeline_stock_balance_{t}",
               mf.Expr.sub(s_p.index(t+1),
                           mf.Expr.sub(mf.Expr.add(s_p.index(t), u_s.index(t)), u_ps.index(t))) == 0)


    for n in range(N):
        # Cancellation must be before t_n[n]
        for t in range(T):
            if t >= t_n[n]:
                M64.constraint(f"cancel_limit_{n}_{t}", y.index(n, t) == 0)

        # x_n = sum y_{n}^t
        M64.constraint(f"cancel_def_{n}",
                     mf.Expr.sub(x.index(n), mf.Expr.sum(y.slice([n, 0], [n+1, T]))) == 0)

        # Cn = sum_t Q_n * f_{n,t} * y_{n}^t
        M64.constraint(f"cancel_cost_{n}",
                     mf.Expr.sub(Cn.index(n),
                                 mf.Expr.dot([Q_n[n] * f_nt[n][t] for t in range(T)],
                                             y.slice([n, 0], [n+1, T]))) == 0)

        # Fn = c_{t_n} * s_n[n][t_n]
        arrival = t_n[n] - 1
        M64.constraint(f"arrival_cost_{n}",
                     mf.Expr.sub(Fn.index(n), mf.Expr.mul(c_t[arrival], s_n.index(n, arrival))) == 0)

    # Solving the model
    M64.solve()
    
    solution_status = M64.getPrimalSolutionStatus()
    minimal_cost = M64.primalObjValue()

    # Display solution
    print("Solution status:", solution_status)
    print("\nMinimal cost:", minimal_cost)
    print("\nOptimal quantities of gas injected in the storage:")
    for t in range(T):
        print(f"u_s[{t}] = {u_s.index(t).level()[0]}")

    print("\nOptimal quantities of gas injected in the pipeline:")
    for t in range(T):
        print(f"u_p[{t}] = {u_p.index(t).level()[0]}")

    print("\nOptimal quantities of gas transferred from the pipeline to the clients:")
    for t in range(T):
        print(f"u_ps[{t}] = {u_ps.index(t).level()[0]}")

    print("\nOptimal quantities of unsatisfied demand:")
    for t in range(T):
        print(f"u_NS[{t}] = {u_NS.index(t).level()[0]}")

    print("\nPipeline storage at the beginning of time t:")
    for t in range(T+1):
        print(f"s_p[{t}] = {s_p.index(t).level()[0]}")

    print("\nStock in ship of contract n at t >= t_n-1:")
    for n in range(N):
        for t in range(T):
            print(f"s_n[{n},{t}] = {s_n.index(n, t).level()[0]}")

    print("\nCancellation cost at t for contract n:")
    for n in range(N):
        print(f"Cn[{n}] = {Cn.index(n).level()[0]}")
    
    print("\nShip load n cost at t:")
    for n in range(N):
        print(f"Fn[{n}] = {Fn.index(n).level()[0]}")

    print("\nCancellation status of contracts:")
    for n in range(N):
        print(f"Contract {n+1} cancelled: {'Yes' if x.index(n).level()[0] == 1 else 'No'}")
        if x.index(n).level()[0] == 1:
            print(f"Cancelled at time t_n[{n}] = {t_n[n]}")
        else:
            print(f"Contract {n+1} is active.")

    print("\nCancellation status of contracts at each time t:")
    for n in range(N):
        for t in range(T):
            print(f"y[{n},{t}] = {y.index(n, t).level()[0]}")



SolutionError: Solution status is Unknown but Optimal is expected. Reason: Accessing integer solution whose problem status is PrimalInfeasibleOrUnbounded.

### Exercise 5

In [111]:
# The Backpack problem - List 6 Exercise 5

with mf.Model("list6_exercise5") as M65:
    # Paremeters
    # Number of items
    N = 25
    # Maximum weight of the backpack
    W = 10
    # Maximum volume of the backpack
    V = 5
    # Weights of the items
    weight = [3, 2, 1, 0.5, 0.2, 4, 0.5, 0.3, 2, 0.3, 0.1, 0.1, 1.5, 0.1, 0.3, 0.2, 0.5, 0.1, 0.5,
    0.3, 0.2, 0.3, 0.2, 2, 0.5]
    # Utilities of the items
    utility = [9, 10, 8, 7, 6, 9, 9, 8, 10, 9, 8, 7, 9, 7, 8, 7, 9, 7, 10, 8, 7, 8, 7, 8, 9]
    # Volumes of the items
    volume = [0.5, 0.4, 0.3, 0.2, 0.1, 0.4, 0.2, 0.1, 0.3, 0.05, 0.6, 0.2, 0.15, 0.4, 0.3, 0.2, 0.1, 0.4, 0.5, 0.3, 0.2, 0.4, 0.5, 0.7, 0.2]

    # Decision variables
    # x[i] = 1 if item i is selected and 0 otherwise
    # x[i] >= 0 and x is an integer
    x = M65.variable("x", [N], mf.Domain.binary())

    # Objective function: maximize total utility
    M65.objective("MaximizeUtility", mf.ObjectiveSense.Maximize, mf.Expr.dot(utility, x))

    # Constraint: total weight of the selected items
    M65.constraint("WeightConstraint", mf.Expr.dot(weight, x), mf.Domain.lessThan(W))

    # Constraint: total volume of the selected items
    M65.constraint("VolumeConstraint", mf.Expr.dot(volume, x), mf.Domain.lessThan(V))

    # Solving the model
    M65.solve()

    solution_status = M65.getPrimalSolutionStatus()
    maximal_utility = M65.primalObjValue()
    optimal_x = [int(round(x.index(i).level()[0])) for i in range(N)]

    # Display solution
    print("Solution status:", solution_status)
    print("\nMaximal utility:", maximal_utility)
    print("\nTotal weight of the selected items:", round(sum(weight[i] for i in range(N) if optimal_x[i] == 1),2))
    print("\nTotal volume of the selected items:", sum(volume[i] for i in range(N) if optimal_x[i] == 1))
    # Display matrix of items with their weight, utility, and selection status
    print("\nItem | Weight | Volume | Utility | Selected")
    print("---------------------------------------------")
    for i in range(N):
        print(f"{i+1:4} | {weight[i]:6.2f} | {volume[i]:6.2f} | {utility[i]:7} | {'Yes' if optimal_x[i] == 1 else 'No'}")

Solution status: SolutionStatus.Optimal

Maximal utility: 154.0

Total weight of the selected items: 9.5

Total volume of the selected items: 4.9

Item | Weight | Volume | Utility | Selected
---------------------------------------------
   1 |   3.00 |   0.50 |     9.0 | No
   2 |   2.00 |   0.40 |    10.0 | No
   3 |   1.00 |   0.30 |     8.0 | Yes
   4 |   0.50 |   0.20 |     7.0 | Yes
   5 |   0.20 |   0.10 |     6.0 | Yes
   6 |   4.00 |   0.40 |     9.0 | No
   7 |   0.50 |   0.20 |     9.0 | Yes
   8 |   0.30 |   0.10 |     8.0 | Yes
   9 |   2.00 |   0.30 |    10.0 | Yes
  10 |   0.30 |   0.05 |     9.0 | Yes
  11 |   0.10 |   0.60 |     8.0 | Yes
  12 |   0.10 |   0.20 |     7.0 | Yes
  13 |   1.50 |   0.15 |     9.0 | Yes
  14 |   0.10 |   0.40 |     7.0 | No
  15 |   0.30 |   0.30 |     8.0 | Yes
  16 |   0.20 |   0.20 |     7.0 | Yes
  17 |   0.50 |   0.10 |     9.0 | Yes
  18 |   0.10 |   0.40 |     7.0 | No
  19 |   0.50 |   0.50 |    10.0 | Yes
  20 |   0.30 |   0.30 |   

### Exercise 6

In [110]:
# Unit Commitment Problem - List 6 Exercise 6

with mf.Model("list6_exercise6") as M66:
    # Indexes
    # Number of thermal plants
    N = 10 
    # Number of months
    T = 12

    # Paremeters
    # Monthly demand
    D = [60000, 58000, 55000, 50000, 48000, 45000, 55000, 58000, 60000, 58000, 55000, 50000]
    # Production linear cost for each plant
    c = [0.8, 0.85, 0.9, 1.0, 1.05, 1.1, 1.2, 1.25, 1.3, 1.35]
    # Cost of turning on each plant
    p = [1500, 1400, 1300, 1200, 1100, 1000, 900, 800, 700, 600]
    # Maximum production capacity for each plant per month
    K = [12000, 12000, 12000, 12000, 12000, 8000, 8000, 8000, 8000, 8000]

    
    # Decision variables
    # y[i,t] = 1 if plant i is turned on in month t and 0 otherwise
    # y[i,t] >= 0 and y[i,t] is binary
    y = M66.variable("y", [N, T], mf.Domain.binary())
    # u[i,t] = amount of electricity produced by plant i in month t
    # u[i,t] >= 0 and u[i,t] is an integer
    u = M66.variable("u", [N, T], mf.Domain.integral(mf.Domain.greaterThan(0.0)))

    # Objective function: minimize total cost
    # First term: production cost sum_{i,t} c_i * u_{i,t}
    production_cost = mf.Expr.add([mf.Expr.mul(c[i], u.index([i, t])) for i in range(N) for t in range(T)])

    # Second term: startup cost for the first month and the other months
    # First month (t=0): sum_i p_i * y_{i,0}
    startup_cost = mf.Expr.add([mf.Expr.mul(p[i], y.index([i, 0])) for i in range(N)])
    # Subsequent months (t>=1): sum_{i,t} p_i * (y_{i,t} - y_{i,t-1})
    for i in range(N):
        for t in range(1, T):
            # Startup cost for the sybsequent months (if y[i,t] - y[i,t-1] = 1)
            startup_cost = mf.Expr.add(startup_cost, mf.Expr.mul(p[i], mf.Expr.sub(y.index([i, t]), y.index([i, t-1]))))

    total_cost = mf.Expr.add(production_cost, startup_cost)
    M66.objective("MinimizeCost", mf.ObjectiveSense.Minimize, total_cost)

    # Contraint: once a plant is turned on, it remains on
    for i in range(N):
        for t in range(1, T):
            M66.constraint(mf.Expr.sub(y.index([i, t]), y.index([i, t-1])), mf.Domain.greaterThan(0.0))

    # Constraint: production limits 0 <= u[i,t] <= y[i,t] * K[i]
    for i in range(N):
        for t in range(T):
            M66.constraint(mf.Expr.sub(u.index([i, t]), mf.Expr.mul(K[i], y.index([i, t]))), mf.Domain.lessThan(0.0))

    # Constraint: demand satisfaction sum_i u[i,t] = D[t]
    for t in range(T):
        M66.constraint(mf.Expr.add([u.index([i, t]) for i in range(N)]), mf.Domain.equalsTo(D[t]))

    # Solving the model
    M66.solve()

    solution_status = M66.getPrimalSolutionStatus()

    # Display solution
    print("Solution status:", solution_status)

    if M66.getPrimalSolutionStatus() == mf.SolutionStatus.Optimal:
        print("\nMinimal cost:", M66.primalObjValue())
        
        y_val = y.level().reshape(N, T)
        u_val = u.level().reshape(N, T)
        
        print("\nTurned on thermal plants and their montlhy production:")
        for i in range(N):
            if sum(y_val[i, :]) > 0:
                print(f"Plant {i+1}:")
                print(f"  - On since month {[t+1 for t in range(T) if y_val[i,t] == 1 and (t == 0 or y_val[i,t-1] == 0)]}")
                print(f"  - Monthly production:", [round(u_val[i, t], 1) for t in range(T)])


Solution status: SolutionStatus.Optimal

Minimal cost: 597650.0

Turned on thermal plants and their montlhy production:
Plant 1:
  - On since month [1]
  - Monthly production: [12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0]
Plant 2:
  - On since month [1]
  - Monthly production: [12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0]
Plant 3:
  - On since month [1]
  - Monthly production: [12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0]
Plant 4:
  - On since month [1]
  - Monthly production: [12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 9000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0, 12000.0]
Plant 5:
  - On since month [1]
  - Monthly production: [12000.0, 10000.0, 7000.0, 2000.0, 0.0, 0.0, 7000.0, 10000.0, 12000.0, 10000.0, 7000.0, 2000.0]
